In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/leapGestRecog/00'
os.listdir(dataset_path)

['01_palm',
 '08_palm_moved',
 '06_index',
 '10_down',
 '09_c',
 '04_fist_Moved',
 '07_ok',
 '05_thumb',
 '03_fist',
 '02_I']

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical


In [8]:
DATASET_PATH = "/content/drive/MyDrive/Colab Notebooks/Dataset/leapGestRecog/00"

IMG_SIZE = 64   # resize images
data = []
labels = []

for folder in os.listdir(DATASET_PATH):
    folder_path = os.path.join(DATASET_PATH, folder)

    if os.path.isdir(folder_path):
        for img_name in os.listdir(folder_path)[:20]:  # take only 20 images
            img_path = os.path.join(folder_path, img_name)

            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

            data.append(img)
            labels.append(folder)

In [9]:
X = np.array(data).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X = X / 255.0  # normalize

y = np.array(labels)


In [10]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42
)


In [12]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(64,64,1)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_categorical.shape[1], activation='softmax'))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,625,866 (6.20 MB)

 Trainable params: 1,625,866 (6.20 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
history = model.fit(
    X_train, y_train,
    epochs=15,
    batch_size=16,
    validation_data=(X_test, y_test)
)


Epoch 1/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 353ms/step - accuracy: 0.1133 - loss: 2.3028 - val_accuracy: 0.4500 - val_loss: 2.0706
Epoch 2/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 0.4353 - loss: 1.9323 - val_accuracy: 0.8750 - val_loss: 1.2700
Epoch 3/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - accuracy: 0.6924 - loss: 1.1960 - val_accuracy: 0.9250 - val_loss: 0.4667
Epoch 4/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - accuracy: 0.8083 - loss: 0.6692 - val_accuracy: 1.0000 - val_loss: 0.1255
Epoch 5/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step - accuracy: 0.9221 - loss: 0.3068 - val_accuracy: 1.0000 - val_loss: 0.0533
Epoch 6/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - accuracy: 0.9336 - loss: 0.1992 - val_accuracy: 1.0000 - val_loss: 0.0347
Epoch 7/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - accuracy: 0.9217 - loss: 0.1970 - val_accuracy: 1.0000 - val_loss: 0.0336
Epoch 8/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - accuracy: 0.9763 - loss: 0.1139 - val_accuracy: 1.

In [14]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 1.0000 - loss: 5.9454e-04
Test Accuracy: 1.0


In [15]:
model.save("/content/drive/MyDrive/hand_gesture_model.h5")


In [20]:
def predict_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    if img is None:
        print(f"Error: Could not load image from {img_path}")
        return "Image not loaded"

    img = cv2.resize(img, (64,64))
    img = img.reshape(1,64,64,1) / 255.0

    prediction = model.predict(img)
    class_index = np.argmax(prediction)

    return label_encoder.inverse_transform([class_index])[0]

In [24]:
test_image = "/content/drive/MyDrive/Colab Notebooks/Dataset/leapGestRecog/00/01_palm/frame_00_01_0001.png"
print("Predicted Gesture:", predict_image(test_image))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
Predicted Gesture: 01_palm
